In [ ]:
%pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import re
dataset = pd.read_csv('imp_final_data.csv',encoding = 'unicode_escape')

In [ ]:
dataset['QueryType'] = dataset['QueryType'].str.replace(r'\t', '')

<ipython-input-22-bd6be94d755b>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['QueryType'] = dataset['QueryType'].str.replace(r'\t', '')


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('all-mpnet-base-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
import requests, json
# API base URL
BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"

def get_weather_info(CITY):

     API_KEY = "9a86fa9066224859399de5ee1bb8602e"
     URL = BASE_URL + "q=" + CITY + "&appid=" + API_KEY
     response = requests.get(URL)

     if response.status_code == 200:
    
       data = response.json()
       main = data['main']
       temperature = main['temp']
       temp_feel_like = main['feels_like']  
       humidity = main['humidity']
       pressure = main['pressure']
       weather_report = data['weather']
       wind_report = data['wind']
   
       print(f"{CITY:-^35}")
       print(f"City ID: {data['id']}")

       temp = float(temperature) 
       f = ((temp - 273.15) * 9/5)+32
       celsius = (f - 32) * 5/9
       feels_like = float(temp_feel_like)
       f1 = ((feels_like - 273.15) * 9/5)+32
       celsius1 = (f1 - 32) * 5/9
       print("")
       print("====================================")
       print(f"Temperature:" "{0:.2f}°C".format(celsius))
       print(f"Feel Like:" "{0:.2f}°C".format(celsius1))    
       print(f"Humidity: {humidity} %")
       print(f"Atmospheric Pressure: {pressure} hPa")
       print(f"Weather Report: {weather_report[0]['description']}")
       print(f"Wind Speed: {wind_report['speed']} km/h")
       print("====================================")
       print("")
     else:
       print("Error in the HTTP request")

In [ ]:
import requests
res = requests.get("https://api.data.gov.in/resource/9ef84268-d588-465a-a308-a864a43d0070?api-key=579b464db66ec23bdd0000014300ea7a708648397e8195bb436adc38&format=json&limit=1000")
all_data=res.json()
for y in all_data["records"][:2]:
  for k in y:
    print(k,y[k])

state Bihar
district Bhagalpur
market Bhagalpur
commodity Cabbage
variety Cabbage
grade FAQ
arrival_date 14/01/2023
min_price 1600
max_price 1900
modal_price 1800
state Bihar
district East Champaran/ Motihari
market Motihari
commodity Cauliflower
variety African Sarson
grade FAQ
arrival_date 14/01/2023
min_price 2000
max_price 2500
modal_price 2200


In [ ]:
def get_answer_database(query_type, query,crop,category):
    """
        returns the answer from the database based on query
    """
    subset_data = dataset.loc[dataset['Category'] == category_dict[category]]
    subset_data = subset_data.loc[subset_data['Crop'] == crop_dict[crop]]
    subset_data = subset_data.loc[subset_data['QueryType'] == query_type_dict[query_type]]
    dataset_dict = subset_data.to_dict('list')
    dataset_temp = pd.DataFrame.from_dict(dataset_dict)
    questions = subset_data['QueryText'].to_list()
    # Single list of sentences
    sentences = questions
    #Compute embeddings
    embeddings_ques = model.encode(questions, convert_to_tensor=True)
    embedding_query = model.encode(query, convert_to_tensor=True)

    #Compute cosine-similarities for each sentence with each other sentence
    cosine_scores = util.cos_sim(embeddings_ques, embedding_query)
    results = cosine_scores.reshape(1, -1)
    res_index = np.argmax(results[0]).item()
    res_index
    print(f"Similarity Score of the top resulting question: {results[0][res_index]}")
     
    THRESHOLD = 0.3
    if results[0][res_index] > THRESHOLD:
        # Respective question found on the database
        print("========================================")
        print(f"Question: {query}")
        print(f"Question on Database: {dataset_temp['QueryText'][res_index]}")
        print("========================================")
        print(f"Answer: {dataset_temp['KccAns'][res_index]}")
        print("========================================")
        print(f"Reference: {dataset_temp['Refer'][res_index]}")
        print("========================================")
    elif results[0][res_index]>=0 and results[0][res_index]<=THRESHOLD:
        print("We find some similar answer for your question.Thank You ")
        print(f"Question on Database: {dataset_temp['QueryText'][res_index]}")
        print("========================================")
        print(f"Answer: {dataset_temp['KccAns'][res_index]}")
        print("========================================")
        print(f" {dataset_temp['Refer'][res_index]}")    
    else:
        print("========================================")
        print(f"Sorry.We are not find your question in our database. But we also answer various types of queries like 1.Plant Protection 2.Cultural Practices 3.Government Schemes 4.Nutrient Management 5.Fertilizer Uses 6.Varieties 7.Weed Management 8.Others 9.Seeds 10.Water Management")

In [ ]:
category_dict = {"1": "Todays Weather","2": "Mandi Rate","3": "Vegetables","4":"Spices","5": "Schemes","6": "Communication","7": "Fruits","8": "Cereals",
                   "9": "Oilseeds","10": "Pulses"}

In [ ]:
query_type_dict = {"1": "Cultural Practices","2": "Government Schemes","3": "Nutrient Management","4":"Fertilizer Uses","5": "Varieties","6": "Weed Management","7": "Seeds","8": "Contact Number",
                   "9": "Address","10": "Water Management","11": "Plant Protection"}

In [ ]:
crop_dict = {"1": "Ash Gourd","2": "Turnip","3": "Tomato","4": "Sweet Potato","5": "Squash Melon","6": "Sponge Gourd","7": "Spinach (Palak)",
"8": "Radish","9": "Rabi Onion(Pyaz)","10": "Pumpkin","11": "Potato","12": "Peas","13": "Okra",
"14": "Long Melon","15": "Lettuce","16": "Kharif Onion(Pyaz)","17": "Garlic","18": "Cucumber",
"19": "Chilli","20": "Cauliflower","21": "Carrot","22": "Beetroot","23": "Bitter Gourd","24": "Capsicum","25": "Cabbage","26": "Broccoli","27": "Brinjal","28": "Coriander","29": "Ginger",
"30": "Turmeric","31": "Pradhan Mantri KISAN Samman Nidhi Yojana","32": "Pradhan Mantri Krishi Sinchai Yojana (PMKSY)","33": "Saur Sinchai Yojana",
"34": "Rashtriya Krishi Vikas Yojana","35": "Pradhan Mantri Fasal Bima Yojana (PMFBY)",
"36": "Pradhan Mantri Kisan Maan-Dhan Yojana","37": "Soil Health Card Scheme","38": "PM Kusum Yojana",
"39": "Pardhan Mantri Kisan SAMPADA Yojana","40": "Machinery/Equipment Subsidy",
"41": "Lemon","42": "Lime","43": "Banana","44": "Ber","45": "Grapes","46": "Guava","47": "Jamun",
"48": "Litchi","49": "Mango","50": "Orange/Santra","51": "WaterMelon","52": "Malta","53": "Papaya",
"54": "Barley(Jow)","55": "Maize(Kharif)","56": "Oats","57": "Wheat",
"58": "Rice","59": "Sugarcane","60": "Groundnut","61": "Mustard",
"62": "Soybean","63": "Sunflower","64": "Lentil","65": "Bengal Gram","66": "Green Gram","67": "Kidney Bean","68": "Mash",
"69": "Pigeon Pea","70": "RiceBean","71": "Institution","72": "Krishi Vigyan Kendras","73": "Nodal Officer Details (Statewise)"
             }

In [ ]:
%pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()

def spell_check(sentence):
    corrected_sentence = ""
    words = sentence.split(" ")
    for word in words:
        corrected_word = spell.correction(word)
        corrected_sentence += corrected_word + " "
    return corrected_sentence

#sentence = input("Enter a sentence: ")
#print("Corrected sentence: ", spell_check(sentence))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def get_name():
    print("===================================")
    print("Welcome to the fasalonline chatbot")
    print("===================================")
    print("I am here to help you out.")
    print('May I know your name?')
    name = input("Name:")
    print(" ")
    print("\nSo %s, How can I help you?, Please select category from the option" %(name))


def get_response():
    """
        Main Function to get queries answer from the user
    """
    print("===================================")
    print("Reply 1 for Todays Weather")
    print("Reply 2 for Mandi Rate")
    print("Reply 3 for Vegetables")
    print("Reply 4 for Spices")
    print("Reply 5 for Schemes")
    print("Reply 6 for Communication")
    print("Reply 7 for Fruits")
    print("Reply 8 for Cereals")
    print("Reply 9 for Oilseeds")
    print("Reply 10 for Pulses")
    print("===================================")

    # Category type
    category = input(str("Enter Category type: "))

    # weather 
    if category == "1":
      CITY = input("Enter the city: ")
      get_weather_info(CITY)

    #government scheme  
    elif category == "5":
        print("")
        print("===================================")
        print("Reply 2 for Government Schemes")
        print("===================================")
        print("")
        query_type = input(str("Enter query type: "))
        print("")
        print("===================================")
        print("Reply 31 for Pradhan Mantri KISAN Samman Nidhi Yojana")
        print("Reply 32 for Pradhan Mantri Krishi Sinchai Yojana (PMKSY)")
        print("Reply 33 for Saur Sinchai Yojana")
        print("Reply 34 for Rashtriya Krishi Vikas Yojana")
        print("Reply 35 for Pradhan Mantri Fasal Bima Yojana (PMFBY)")
        print("Reply 36 for Pradhan Mantri Kisan Maan-Dhan Yojana")
        print("Reply 37 for Soil Health Card Scheme")
        print("Reply 38 for PM Kusum Yojana")
        print("Reply 39 for Pardhan Mantri Kisan SAMPADA Yojana")
        print("Reply 40 for Machinery/Equipment Subsidy")
        print("===================================")
        print("")
        crop = input(str("Enter the Scheme Name: ")) 
        query = input("Enter your query: ")  
        get_answer_database(query_type,query,crop,category)

    #contact  
    elif category == "6":
      print("")
      print("===================================")
      print("Reply 8 for Contact Number")
      print("Reply 9 for Address")
      print("===================================")
      print("")
      query_type = input(str("Enter query type: "))
      print("")
      print("===================================")
      print("Reply 71 for Institution")
      print("Reply 72 for Krishi Vigyan Kendras")
      print("Reply 73 for Nodal Officer Details (Statewise)")
      print("===================================")
      print("")
      crop = input(str("Enter the place Name: ")) 
      query = input("Enter your query: ")  
      get_answer_database(query_type, query,crop,category)

    #vegetables  
    elif category == "3":
      print("")
      print("===================================")
      print("Reply 1 for Ash Gourd")
      print("Reply 2 for Turnip")
      print("Reply 3 for Tomato")
      print("Reply 4 for Sweet Potato")
      print("Reply 5 for Squash Melon")
      print("Reply 6 for Sponge Gourd")
      print("Reply 7 for Spinach (Palak)")
      print("Reply 8 for Radish)")
      print("Reply 9 for Rabi Onion(Pyaz)")
      print("Reply 10 for Pumpkin")
      print("Reply 11 for Potato")
      print("Reply 12 for Peas")
      print("Reply 13 for Okra")
      print("Reply 14 for Long Melon")
      print("Reply 15 for Lettuce")
      print("Reply 16 for Kharif Onion(Pyaz)")
      print("Reply 17 for Garlic")
      print("Reply 18 for Cucumber")
      print("Reply 19 for Chilli")
      print("Reply 20 for Cauliflower")
      print("Reply 21 for Carrot")
      print("Reply 22 for Beetroot")
      print("Reply 23 for Bitter Gourd")
      print("Reply 24 for Capsicum")
      print("Reply 25 for Cabbage")
      print("Reply 26 for Broccoli")
      print("Reply 27 for Brinjal")
      print("===================================")
      print("")
      crop = input(str("Enter the crop Name: "))
      print("")
      print("===================================")
      print("Reply 1 for Cultural Practices")
      print("Reply 3 for Nutrient Management")
      print("Reply 4 for Fertilizer Uses")
      print("Reply 5 for Varieties")
      print("Reply 6 for Weed Management")
      print("Reply 7 for Seeds")
      print("Reply 10 for Water Management")
      print("Reply 11 for Plant Protection ")
      print("===================================")
      print("")
      query_type = input(str("Enter query type: "))
      query = input("Enter your query: ")  
      get_answer_database(query_type, query,crop,category)


    #Condiments   
    elif category == "4":
      print("")
      print("===================================")
      print("Reply 28 for Coriander")
      print("Reply 29 for Ginger")
      print("Reply 30 for Turmeric")
      print("===================================")
      print("")
      crop = input(str("Enter the crop Name: "))
      print("")
      print("===================================")
      print("Reply 1 for Cultural Practices")
      print("Reply 3 for Nutrient Management")
      print("Reply 4 for Fertilizer Uses")
      print("Reply 5 for Varieties")
      print("Reply 6 for Weed Management")
      print("Reply 7 for Seeds")
      print("Reply 10 for Water Management")
      print("Reply 11 for Plant Protection ")
      print("===================================")
      print("")
      query_type = input(str("Enter query type: "))
      query = input("Enter your query: ")  
      get_answer_database(query_type, query,crop,category)

    
    #Fruits   
    elif category == "7":
      print("")
      print("===================================")
      print("Reply 41 for Lemon")
      print("Reply 42 for Lime")
      print("Reply 43 for Banana")
      print("Reply 44 for Ber")
      print("Reply 45 for Grapes")
      print("Reply 46 for Guava")
      print("Reply 47 for Jamun")
      print("Reply 48 for Litchi")
      print("Reply 49 for Mango")
      print("Reply 50 for Orange/Santra")
      print("Reply 51 for WaterMelon")
      print("Reply 52 for Malta")
      print("Reply 53 for Papaya")
      print("===================================")
      print("")
      crop = input(str("Enter the crop Name: "))
      print("")
      print("===================================")
      print("Reply 1 for Cultural Practices")
      print("Reply 3 for Nutrient Management")
      print("Reply 4 for Fertilizer Uses")
      print("Reply 5 for Varieties")
      print("Reply 6 for Weed Management")
      print("Reply 7 for Seeds")
      print("Reply 10 for Water Management")
      print("Reply 11 for Plant Protection ")
      print("===================================")
      print("")
      query_type = input(str("Enter query type: "))
      query = input("Enter your query: ")  
      get_answer_database(query_type, query,crop,category)

    #Cereals   
    elif category == "8":
      print("")
      print("===================================")
      print("Reply 54 for Barley(Jow)")
      print("Reply 55 for Maize(Kharif)")
      print("Reply 56 for Oats")
      print("Reply 57 for Wheat")
      print("Reply 58 for Rice")
      print("Reply 59 for Sugarcane")
      print("===================================")
      print("")
      crop = input(str("Enter the crop Name: "))
      print("")
      print("===================================")
      print("Reply 1 for Cultural Practices")
      print("Reply 3 for Nutrient Management")
      print("Reply 4 for Fertilizer Uses")
      print("Reply 5 for Varieties")
      print("Reply 6 for Weed Management")
      print("Reply 7 for Seeds")
      print("Reply 10 for Water Management")
      print("Reply 11 for Plant Protection ")
      print("===================================")
      print("")
      query_type = input(str("Enter query type: "))
      query = input("Enter your query: ")  
      get_answer_database(query_type, query,crop,category)


    #Oil seeds   
    elif category == "9":
      print("")
      print("===================================")
      print("Reply 60 for Groundnut")
      print("Reply 61 for Mustard")
      print("Reply 62 for Soybean")
      print("Reply 63 for Sunflower")
      print("===================================")
      print("")
      crop = input(str("Enter the crop Name: "))
      print("")
      print("===================================")
      print("Reply 1 for Cultural Practices")
      print("Reply 3 for Nutrient Management")
      print("Reply 4 for Fertilizer Uses")
      print("Reply 5 for Varieties")
      print("Reply 6 for Weed Management")
      print("Reply 7 for Seeds")
      print("Reply 10 for Water Management")
      print("Reply 11 for Plant Protection ")
      print("===================================")
      print("")
      query_type = input(str("Enter query type: "))
      query = input("Enter your query: ")  
      get_answer_database(query_type, query,crop,category)



    #Pulses   
    elif category == "10":
      print("")
      print("===================================")
      print("Reply 64 for Lentil")
      print("Reply 65 for Bengal Gram")
      print("Reply 66 for Green Gram")
      print("Reply 67 for Kidney Bean")
      print("Reply 68 for Mash")
      print("Reply 69 for Pigeon Pea")
      print("Reply 70 for RiceBean")
      print("===================================")
      print("")
      crop = input(str("Enter the crop Name: "))
      print("")
      print("===================================")
      print("Reply 1 for Cultural Practices")
      print("Reply 3 for Nutrient Management")
      print("Reply 4 for Fertilizer Uses")
      print("Reply 5 for Varieties")
      print("Reply 6 for Weed Management")
      print("Reply 7 for Seeds")
      print("Reply 10 for Water Management")
      print("Reply 11 for Plant Protection ")
      print("===================================")
      print("")
      query_type = input(str("Enter query type: "))
      query = input("Enter your query: ")  
      get_answer_database(query_type, query,crop,category)

    #market price      
    elif category == "2":
      crop = input(str("Enter the Crop Name: "))
      updated_crop = spell_check(crop)
      updated_crop = updated_crop.strip()
      #crop.lower()
      for x in all_data["records"]:
        if x['commodity'].lower() == updated_crop.lower():
          print(x['commodity'].lower())
          print("Crop_Name = ",x['commodity']," ""State = ",x['state']," " "District=",x['district']," " "Market Name=",x['market']," " "Minimum_price=",x['min_price']," " "Maximum_price=",x['max_price']," " "Modal_price=",x['modal_price'])
    else:
        print("Sorry Try Again")
        print("===================================")
     

In [ ]:
get_name()
while True:
  get_response()
  choice = input("Want to ask more, if yes then type 1 otherwise end the chat to type 2 (1/2)? ")
  if choice == '2':
    print("=======================")
    print("See you soon.Thank You")
    print("=======================")
    break

Welcome to the fasalonline chatbot
I am here to help you out.
May I know your name?
Name:Rahul
 

So Rahul, How can I help you?, Please select category from the option
Reply 1 for Todays Weather
Reply 2 for Mandi Rate
Reply 3 for Vegetables
Reply 4 for Spices
Reply 5 for Schemes
Reply 6 for Communication
Reply 7 for Fruits
Reply 8 for Cereals
Reply 9 for Oilseeds
Reply 10 for Pulses
Enter Category type: 5

Reply 2 for Government Schemes

Enter query type: 2

Reply 31 for Pradhan Mantri KISAN Samman Nidhi Yojana
Reply 32 for Pradhan Mantri Krishi Sinchai Yojana (PMKSY)
Reply 33 for Saur Sinchai Yojana
Reply 34 for Rashtriya Krishi Vikas Yojana
Reply 35 for Pradhan Mantri Fasal Bima Yojana (PMFBY)
Reply 36 for Pradhan Mantri Kisan Maan-Dhan Yojana
Reply 37 for Soil Health Card Scheme
Reply 38 for PM Kusum Yojana
Reply 39 for Pardhan Mantri Kisan SAMPADA Yojana
Reply 40 for Machinery/Equipment Subsidy

Enter the Scheme Name: 40
Enter your query: regarding the machinery scheme
Similarity S